In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Data

In [8]:
df = pd.read_csv('data_20180119_20180420.csv', index_col=0)
df.head()

,t0,T1,K1,C1,T2,K2,C2,S0,Adj_S0
0,2018-01-02,2018-01-19,1000.0,191.525,2018-04-20,800.0,396.725000,59.4505,1189.01001
1,2018-01-02,2018-01-19,1100.0,92.525,2018-04-20,1015.0,191.675000,59.4505,1189.01001
2,2018-01-02,2018-01-19,1150.0,46.075,2018-04-20,1025.0,183.025000,59.4505,1189.01001
3,2018-01-02,2018-01-19,1160.0,38.000,2018-04-20,1050.0,157.776341,59.4505,1189.01001
4,2018-01-02,2018-01-19,1170.0,30.450,2018-04-20,1080.0,137.850000,59.4505,1189.01001


In [9]:
df.Adj_S0.unique()

array([1189.01000977, 1204.19998169, 1209.58999634, 1229.14001465,
       1246.8699646 , 1252.69996643, 1254.32998657, 1276.67999268,
       1305.20004272, 1304.85992432, 1295.        , 1293.32000732,
       1294.57992554])

In [4]:
t0List = list(df.t0.unique())

# dimension: K[k-th t0][t1 and t2][i-th option with i-th strike]
K = [[np.array(df.loc[df.t0 == t0,'K1']),np.array(df.loc[df.t0==t0,'K2'])] for t0 in t0List]

# dimension: Pi[k-th t0][t1 and t2][i-th option with i-th price]
Pi = [[np.array(df.loc[df.t0 == t0,'C1']),np.array(df.loc[df.t0==t0,'C2'])] for t0 in t0List]

In [5]:
# number of t0
N_t0 = len(t0List)

# number of times (t1, t2)
T = 2

# number of options in hedging
N1, N2 = 20, 20
Size = max(N1, N2)

In [6]:
# Assume marginal distribution is uniform
def margDistr(batch_size):
    data = np.random.uniform(size=(batch_size, T))
    for t in range(T):
        data[:, t] = S_min + (S_max - S_min) * data[:, t] # just a simple test, need further consideration
    return data

In [7]:
# arbitrary
S_max = 2000
S_min = 500
BATCH_SIZE = 1000

S = margDistr(BATCH_SIZE)

In [8]:
Phi = (S[:,1]-S[:,0])
# define the payoff function of the derivative

### Variables

In [9]:
# parameter: d
d = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, 1], dtype=tf.float64),
    trainable = True,
    name = 'd'
)

In [10]:
# parameter: lambda
Lambda = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, T, Size], dtype=tf.float64),
    trainable = True,
    name = 'lambda'
)

In [ ]:
# # parameter: Delta
# Delta = tf.Variable(
#     initial_value = tf.random.normal(shape=[N_t0, T-1], dtype=tf.float64),
#     trainable = True,
#     name = 'Delta'
# )

In [11]:
# neural network: Delta(S_1)
Delta = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(BATCH_SIZE,)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(N_t0, activation='linear', dtype=tf.float64)
])

### The step-by-step computation of the objective function

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2}$

In [ ]:
hedgePrice = []
for n in range(N_t0):
    p = d[n]
    for t in range(T):
        p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
    hedgePrice.append(p)

In [ ]:
# take a look
hedgePrice

$d+\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^++\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^++\Delta_{1}(S_1)\cdot(S_2-S_1)$

In [ ]:
hedgeTerm = []
for n in range(N_t0):
    p = d[n]
    for t in range(T):
        p += tf.reduce_sum(Lambda[n,t,:]*np.maximum(S[:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
#     for t in range(T-1):
#         p += Delta[n,t]*(S[:,t+1]-S[:,t])
    p += Delta(S[:,0].reshape((1,-1)))[0,n]*(S[:,1]-S[:,0])
    hedgeTerm.append(p)

In [ ]:
# take a look
hedgeTerm

$\inf_{h\in\mathcal{H}^m}\int hd\mu_0+\int \beta_\gamma(c-h)d\theta$

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2} + \Gamma\cdot\left[\left(\Phi(S_1,S_2)-d-\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^+-\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^+-\Delta_{1}(S_1)\cdot(S_2-S_1)\right)^+\right]^2$

In [ ]:
# objective function
Price = tf.reduce_mean(hedgePrice, axis=1)

Gamma = 100
diff = Phi - tf.stack(hedgeTerm, axis=0)
Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)

ObjFunc = Price + Gamma*Penalty

In [ ]:
# take a look
Price

In [ ]:
# take a look
Penalty

In [ ]:
# take a look
ObjFunc

### Optimisation

In [12]:
# Combine all the computation as a function
def ObjFunc(d, Lambda, Delta, K, Pi, Phi, S):
    hedgePrice = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
        hedgePrice.append(p)
    
    hedgeTerm = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*np.maximum(S[:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
#         for t in range(T-1):
#             p += Delta[n,t]*(S[:,t+1]-S[:,t])
        p += Delta(S[:,0].reshape((1,-1)))[0,n]*(S[:,1]-S[:,0]) # Delta as a neural network
        hedgeTerm.append(p)
    
    Price = tf.reduce_mean(hedgePrice, axis=1)
    Gamma = 100
    diff = Phi-tf.stack(hedgeTerm, axis=0)
    Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)
    
    return Price + Gamma*Penalty, Price

In [13]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = 0.01)

In [ ]:
# for step in range(10000):
#     with tf.GradientTape() as tape:
#         obj, price = ObjFunc(d, Lambda, Delta, K, Pi, Phi, S)
#     gradients = tape.gradient(obj, [d, Lambda, Delta])
#     optimizer.apply_gradients(zip(gradients, [d, Lambda, Delta]))

In [14]:
nIter = 10000
threshold = 1e-3
prev_price = tf.ones((N_t0,), dtype = tf.float64) * float('inf')

for step in range(nIter):
    if (step+1) % 500 == 0:
        print("Processing progress: %d / %d" %(step+1, nIter))
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(d)
        tape.watch(Lambda)
        for variable in Delta.trainable_variables:
            tape.watch(variable)

        obj, price = ObjFunc(d, Lambda, Delta, K, Pi, Phi, S)
    gradients_d, gradients_Lambda = tape.gradient(obj, [d, Lambda])
    gradients_Delta = tape.gradient(obj, Delta.trainable_variables)
    
    optimizer.apply_gradients(
        zip([gradients_d, gradients_Lambda] + gradients_Delta, [d, Lambda] + Delta.trainable_variables)
    )
    
    diff = tf.abs(price - prev_price)
    if tf.reduce_max(diff) < threshold:
        print("Converged at step", step+1)
        break
    
    prev_price = price
    
    del tape

Processing progress: 500 / 10000
Processing progress: 1000 / 10000
Processing progress: 1500 / 10000
Processing progress: 2000 / 10000
Processing progress: 2500 / 10000
Processing progress: 3000 / 10000
Processing progress: 3500 / 10000
Processing progress: 4000 / 10000
Processing progress: 4500 / 10000
Processing progress: 5000 / 10000
Processing progress: 5500 / 10000
Processing progress: 6000 / 10000
Processing progress: 6500 / 10000
Processing progress: 7000 / 10000
Processing progress: 7500 / 10000
Processing progress: 8000 / 10000
Processing progress: 8500 / 10000
Processing progress: 9000 / 10000
Processing progress: 9500 / 10000
Processing progress: 10000 / 10000


In [15]:
price

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([1846.04158057, 2287.81239117, 1232.17535339, 1640.84158345,
       1943.41340725, 1844.56311771, 1468.06375039, 2685.85338762,
       1538.07631024, 1455.77228534,  586.77604876, 1007.23436402,
       1023.71286479])>

In [16]:
obj

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([  45800.66618212,  309332.08605599,  660210.13423921,
        204526.34046012,  474613.85133449,  275540.83762481,
        783213.11918003, 1207762.6979749 ,  700177.47090859,
       2201075.72219859,  854946.31098821,  799002.38621799,
        484389.06464796])>

In [17]:
d

<tf.Variable 'd:0' shape=(13, 1) dtype=float64, numpy=
array([[ 0.82723749],
       [ 1.27846338],
       [-1.28390829],
       [ 1.64108849],
       [ 1.48160762],
       [ 1.30468823],
       [-0.07723492],
       [-0.1865495 ],
       [ 0.07746713],
       [ 2.82613325],
       [ 0.24593387],
       [ 1.86815251],
       [ 1.76615673]])>

In [18]:
Lambda

<tf.Variable 'lambda:0' shape=(13, 2, 20) dtype=float64, numpy=
array([[[-3.94857550e-01,  3.81436315e-01,  6.30794188e-01,
          7.05266450e-01,  1.98299723e+00,  6.95990007e-01,
         -5.62977974e-01,  1.31768904e+00,  1.57961815e+00,
          8.15769428e-01,  1.21923810e+00,  1.51031783e+00,
          1.73394799e+00,  3.07366378e-01,  2.91920203e-01,
          1.22554135e+00, -4.55773539e-01,  1.47815340e+00,
          4.74966148e-01,  1.53526247e+00],
        [ 1.08618202e+00,  3.02047716e-01,  2.08354406e+00,
          4.98177302e-01,  2.22602044e+00,  7.40569702e-01,
          1.77191428e-01,  9.43560285e-01,  5.89826464e-01,
          3.22999898e-01,  4.09001434e-01, -1.11557779e+00,
          1.72740110e+00, -2.98513785e-02,  4.78586926e-01,
          1.70752156e-01,  2.56557242e+00,  1.10110186e+00,
          4.19947266e-01,  3.56195211e-01]],

       [[ 4.37049943e-01,  2.62590396e+00,  5.26282711e-01,
         -4.61248548e-01, -2.75882335e-01, -6.29292850e-01,
      

In [19]:
Delta(S[:,0].reshape((1,-1)))

<tf.Tensor: shape=(1, 13), dtype=float64, numpy=
array([[0.51462862, 0.54844253, 0.77435069, 0.64216309, 0.4065882 ,
        0.31276193, 0.74667481, 0.6102724 , 0.68174072, 0.97233619,
        0.82401652, 0.45626345, 0.76556161]])>